# Piplines chains together multiple steps so that the output  of each step is used as input to the next step

## Pipelines make it easy to apply the same preprocessing to train and test


# -----------------------------------
## Model Build Without Pipeline

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

In [4]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [5]:
x = df.iloc[::,1:]
y = df.iloc[::,0]

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [6]:
# check missing values
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [7]:
# Appling SimpleImputer for missing values

si_age = SimpleImputer()

# -----------------------------------------------------------------------------

# for S,C,Q (String letters) for Error handling used strategy="most_frequent"
si_embarked = SimpleImputer(strategy="most_frequent")

# -----------------------------------------------------------------------------


xtrain_age = si_age.fit_transform(xtrain[['Age']])
xtest_age = si_age.fit_transform(xtest[['Age']])
 
xtrain_embarked =  si_embarked.fit_transform(xtrain[["Embarked"]])
xtest_embarked =  si_embarked.fit_transform(xtest[["Embarked"]])

In [8]:
# Appyling one hot encoding on sex and embarked column

ohe_sex = OneHotEncoder(sparse=False)
ohe_embarked = OneHotEncoder(sparse=False,handle_unknown="ignore")

xtrain_sex = ohe_sex.fit_transform(xtrain[["Sex"]])
xtrain_embarked = ohe_embarked.fit_transform(xtrain_embarked)


xtest_sex = ohe_sex.fit_transform(xtest[["Sex"]])
xtest_embarked = ohe_embarked.fit_transform(xtest_embarked)
xtrain_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [9]:
# extract other column

xtrain_rem = xtrain.drop(columns=["Sex","Age","Embarked"])
xtest_rem = xtest.drop(columns=["Sex","Age","Embarked"])

In [10]:
# concatination all features

xtrain_transform = np.concatenate((xtrain_rem,xtrain_age,xtrain_sex,xtrain_embarked),axis=1)
xtest_transform = np.concatenate((xtest_rem,xtest_age,xtest_sex,xtest_embarked),axis=1)


In [11]:
# Applying model

clf = DecisionTreeClassifier()
clf.fit(xtrain_transform,ytrain)

DecisionTreeClassifier()

In [12]:
#prediction

ypred = clf.predict(xtest_transform)
ypred

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1])

In [13]:
# accuracy_score
from sklearn.metrics import accuracy_score

accuracy_score(ytest,ypred)

0.7374301675977654

In [14]:
# now save the model in pickle for production use

import pickle

pickle.dump(ohe_sex,open('model/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('model/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('model/clf.pkl',"wb"))

In [15]:
# now use for production

import pickle
import numpy as np

ohe_sex = pickle.load(open('model/ohe_sex.pkl','rb'))
ohe_embarked = pickle.load(open('model/ohe_embarked.pkl','rb'))
clf = pickle.load(open('model/clf.pkl',"rb"))


In [16]:
# Now assume that user enter the value

user_pcale = 2
user_gender = "male"
user_age = 31.0
user_sibdp = 0
user_parch = 0
user_fare = 10.5
user_embarked = 'S'

test_input = np.array([user_pcale , user_gender ,user_age ,user_sibdp ,user_parch, user_fare ,user_embarked],dtype=object).reshape(1,7)
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [17]:
# now we need to same all step e.g encodeing etc in production work

# Applying onehotencoding on sex
test_input_sex = ohe_sex.transform(test_input[:,1].reshape(1,1))
test_input_sex


/home/sweeterror404/.local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[0., 1.]])

In [18]:
# appling onehotencoding on embarked
test_input_embarked = ohe_embarked.transform(test_input[:,-1].reshape(1,1))
test_input_embarked

array([[0., 0., 1.]])

In [19]:
test_input_age = test_input[:,2].reshape(1,1)
test_input_age

array([[31.0]], dtype=object)

In [20]:
# concat colum

test_input_transform = np.concatenate((test_input[:,[0,3,4,5]],test_input_age, test_input_sex , test_input_embarked),axis=1)

In [21]:
# prediction

clf.predict(test_input_transform)

array([1])

# -----------------------------------
## Model Build With Pipeline

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [23]:
df = pd.read_csv("./train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [25]:
x = df.iloc[::,1:]
y = df.iloc[::,0]

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [26]:
xtrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


# Step 1 handling missing value with SimpleImputer

In [27]:
from sklearn.compose import ColumnTransformer


trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')



# Step 2 Applying one hot Encoding

In [28]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

# Step 3 Scaling on all column

In [29]:
trf3 = ColumnTransformer([
    # slice means select apply MinMaxScaler only on the range (0 , 10)
    ('scale',MinMaxScaler(),slice(0,10))
    
])

# Step 4 Feature Selection

In [30]:
trf4 = SelectKBest(score_func=chi2,k=8)

# select Best Features
# k means only select 8 Best columns

# Step 5 Build Model

In [31]:
trf5 = DecisionTreeClassifier()

# Create Pipline

In [32]:
pipe = Pipeline([
    
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("trf4",trf4),
    ("trf5",trf5),
])
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f2a1da6d1f0>)),
                ('trf5', DecisionTreeClassifier())])

# Pipeline vs make_pipline

## Both are same 
   * ### In Pipline required name
        * ### e.g Pipleline( ("name",columntransformer1) , ("name",columntransformer2)....)
   * ### In make_pipline not required name
        * ### e.g make_pipline(colt1,colt2 ...)

## Recommend to use Pipline

# Now Train the model

# Note if we are not add a Algorithm means we not build a model than we are use


## pip.transform()

In [33]:
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f2a1da6d1f0>)),
                ('trf5', DecisionTreeClassifier())])

## Pipline Steps show

In [34]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7f2a1da6d1f0>),
 'trf5': DecisionTreeClassifier()}

# if diagram are not show than use set_config

In [53]:
from sklearn import set_config
set_config(display="diagram")

## Explore Pipline specfic step

In [54]:
x = pipe.named_steps["trf1"]
x.transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [52]:
x.transformers_[0]

('impute_age', SimpleImputer(), [2])

In [38]:
x.transformers_[0][1]

SimpleImputer()

In [39]:
x.transformers_[0][1].strategy

'mean'

In [40]:
x.transformers_[0][1].statistics_

array([29.49884615])

## predection

In [55]:
ypred = pipe.predict(xtest)
ypred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

# cross validation

In [56]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,xtrain,ytrain,cv=5,scoring='accuracy').mean()

0.6391214419383433

# GridSearch 

In [43]:
from sklearn.model_selection import GridSearchCV

para = {
    "trf5__max_depth":[1,2,3,4,5,None]
}

grid = GridSearchCV(pipe, para,cv=5,scoring="accuracy")
grid.fit(xtrain,ytrain)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7f2a1da6d1f0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [44]:
grid.best_score_

0.6391214419383433

In [45]:
grid.best_params_

{'trf5__max_depth': 2}

# Save the Model


In [46]:
import pickle
pickle.dump(pipe,open("model/pip.pkl","wb"))

# use the model in production

In [47]:

pipe = pickle.load(open("model/pip.pkl","rb"))
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f2a1da6d1f0>)),
                ('trf5', DecisionTreeClassifier())])

In [48]:
user_pcale = 2
user_gender = "male"
user_age = 31.0
user_sibdp = 0
user_parch = 0
user_fare = 10.5
user_embarked = 'S'
test_input = np.array([user_pcale , user_gender ,user_age ,user_sibdp ,user_parch, user_fare ,user_embarked],dtype=object).reshape(1,7)
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [50]:
pipe.predict(test_input)

/home/sweeterror404/.local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/home/sweeterror404/.local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])